## CARGA DE LIBRERÍAS

In [2]:
# HTTP requests
import requests
# Date handler utility
import datetime
# Mark-Language parsing
import bs4
# DataFrame Manager
import pandas
# Random
import random
random.seed = 42
# Thread management (sleep purposes)
import time
# Operating System management
import os

import selenium

from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd
import re
import numpy as np

## OBTENCIÓN DE LOS DATOS USANDO WEB SCRAPPING

In [2]:
USER_AGENT = ""

In [3]:
options = webdriver.ChromeOptions()

options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

if USER_AGENT:
  options.add_argument(f"user-agent={USER_AGENT}")

driver = webdriver.Chrome(options=options)
     

In [4]:
driver.implicitly_wait(1)
driver.get("https://www.indomio.es/venta-casas/valencia-capital/")

In [5]:
links = driver.find_elements(By.TAG_NAME, 'a')

In [6]:
valids = []
# Iterate over link elements
for i in range(len(links)):
    href = links[i].get_attribute('href')
    if href and (re.search("^https://www.indomio.es/venta-casas/valencia-capital/[a-zA-Z-]+/$", href) is not None):
        valids.append(href)

In [7]:
links[8].get_attribute('href')

'https://www.indomio.es/venta-casas/valencia-capital/l-olivereta/'

In [8]:
valids = valids[:19]

In [21]:
data = pd.DataFrame(columns = ["Title","Price","Characteristics", "Barrio"])

In [22]:
for link in valids:
    driver.get(link)
    barrio = re.match("^https://www.indomio.es/venta-casas/valencia-capital/([a-zA-Z-]+)/$",link)
    barrio = barrio.group(1)
    price = [element.text for element in driver.find_elements(by = By.CLASS_NAME, value = 'in-reListCardPrice')]
    barrio = list(np.full(len(price), barrio))
    title = [element.text for element in driver.find_elements(by = By.CLASS_NAME, value = 'in-reListCard__title')] 
    characteristics =  [element.text for element in driver.find_elements(by = By.CLASS_NAME, value = 'in-reListCardFeatureList')]
    aux_df = pd.DataFrame({"Title":title, "Price": price, "Characteristics":characteristics, "Barrio":barrio})
    data = pd.concat([data, aux_df], ignore_index=True)
    i = 2
    while len(price) >= 25:
        linknew = link + f'?pag={i}'
        print(linknew)
        driver.get(linknew)
        price = [element.text for element in driver.find_elements(by = By.CLASS_NAME, value = 'in-reListCardPrice')]
        if len(price) < 1:
            break
        barrio = re.match("^https://www.indomio.es/venta-casas/valencia-capital/([a-zA-Z-]+)/$",link)
        barrio = barrio.group(1)
        barrio = list(np.full(len(price), barrio))
        title = [element.text for element in driver.find_elements(by = By.CLASS_NAME, value = 'in-reListCard__title')]  
        characteristics =  [element.text for element in driver.find_elements(by = By.CLASS_NAME, value = 'in-reListCardFeatureList')]
        aux_df = pd.DataFrame({"Title":title, "Price": price, "Characteristics":characteristics, "Barrio":barrio})
        data = pd.concat([data, aux_df], ignore_index=True)
        i += 1
    

https://www.indomio.es/venta-casas/valencia-capital/sant-isidre-patraix-favara-vara-de-quart/?pag=2
https://www.indomio.es/venta-casas/valencia-capital/sant-isidre-patraix-favara-vara-de-quart/?pag=3
https://www.indomio.es/venta-casas/valencia-capital/l-eixample/?pag=2
https://www.indomio.es/venta-casas/valencia-capital/l-eixample/?pag=3
https://www.indomio.es/venta-casas/valencia-capital/l-eixample/?pag=4
https://www.indomio.es/venta-casas/valencia-capital/l-eixample/?pag=5
https://www.indomio.es/venta-casas/valencia-capital/l-eixample/?pag=6
https://www.indomio.es/venta-casas/valencia-capital/l-olivereta/?pag=2
https://www.indomio.es/venta-casas/valencia-capital/l-olivereta/?pag=3
https://www.indomio.es/venta-casas/valencia-capital/l-olivereta/?pag=4
https://www.indomio.es/venta-casas/valencia-capital/rascanya/?pag=2
https://www.indomio.es/venta-casas/valencia-capital/rascanya/?pag=3
https://www.indomio.es/venta-casas/valencia-capital/el-pla-del-real/?pag=2
https://www.indomio.es/ven

In [24]:
driver.close()

In [27]:
pd.DataFrame.to_csv(data, "data/WS_Indomio.csv")

## PRE-PRPCESADO DE LOS DATOS

In [2]:
data = pd.read_csv("data/WS_Indomio.csv")

In [5]:
new_columns = data['Characteristics'].str.split('\n', expand=True)
column_names = ['hab', 'm2', 'bath', 'charact1', 'charact2', 'charact3']
new_columns.columns = column_names
data = pd.concat([data, new_columns], axis=1)
data.drop('Characteristics', axis=1, inplace=True)
data

,Title,Price,Barrio,hab,m2,bath,charact1,charact2,charact3
0,"Piso de tres habitaciones Segura Lago, Sant Is...",€ 159.000,sant-isidre-patraix-favara-vara-de-quart,3 habitaciones\r,91 m²\r,1 baño\r,Planta 3\r,Ascensor\r,Balcón
1,"Piso de tres habitaciones Gremis Dels, Sant Is...",€ 245.000,sant-isidre-patraix-favara-vara-de-quart,3 habitaciones\r,132 m²\r,2 baños\r,Ascensor\r,Balcón\r,Terraza
2,"Ático Calle CERAMISTA ROS, Vara de Quart, Valè...",€ 210.000,sant-isidre-patraix-favara-vara-de-quart,2 habitaciones\r,55 m²\r,1 baño\r,Planta 8\r,Ascensor,None
3,"Piso tercera planta, Patraix, València",€ 400.000,sant-isidre-patraix-favara-vara-de-quart,5 habitaciones\r,151 m²\r,2 baños\r,Planta 3\r,Ascensor\r,Balcón
4,"Ático 115 m² con terraza, La Raiosa, València",€ 112.500,sant-isidre-patraix-favara-vara-de-quart,1 habitación\r,115 m²\r,1 baño\r,Planta 5\r,Balcón\r,Terraza
...,...,...,...,...,...,...,...,...,...
1179,"Piso de cuatro habitaciones buen estado, Penya...",€ 1.200.000,camins-al-grau,4 habitaciones\r,175 m²\r,3 baños\r,Lujo\r,Ascensor\r,Terraza
1180,"Piso Calle Padre Tomás Montañana 10, Camí Fond...","€ 215.000\r\n€ 230.000(-6,5%)",camins-al-grau,3 habitaciones\r,86 m²\r,2 baños\r,Planta 4\r,Ascensor\r,Balcón
1181,"Piso de tres habitaciones 86 m², Camí Fondo, V...",€ 230.000,camins-al-grau,3 habitaciones\r,86 m²\r,2 baños\r,Planta 4\r,Ascensor\r,Balcón
1182,"Piso de dos habitaciones séptima planta, Aiora...",€ 182.500,camins-al-grau,2 habitaciones\r,80 m²\r,2 baños\r,Planta 7\r,Ascensor,None


In [6]:
pd.DataFrame.to_csv(data, "data/WS_Indomio.csv")

In [11]:
data = pd.read_csv("data/WS_Indomio.csv", index_col=0)

In [13]:
import pandas as pd

def eliminar_filas_sin_palabras(df, columna, palabras):
    """
    Elimina las filas del DataFrame que no contienen ninguna de las palabras especificadas en la columna dada.
    
    Args:
    - df: DataFrame a modificar.
    - columna: Nombre de la columna en la que se buscan las palabras.
    - palabras: Lista de palabras que deben estar presentes en las filas que se mantendrán.
    
    Returns:
    - DataFrame modificado sin las filas que no contienen ninguna de las palabras.
    """
    # Eliminar filas con valores NaN en la columna especificada
    df = df.dropna(subset=[columna])
    
    # Aplicar el filtro para mantener las filas que contienen alguna de las palabras
    filtro = df[columna].str.contains('|'.join(palabras), case=False)
    return df[filtro]


# Llamar a la función para mantener solo las filas que contienen 'alquiler' en la columna 'descripcion'
data = eliminar_filas_sin_palabras(data, 'hab', ["habitaciones", "habitación"])
data = eliminar_filas_sin_palabras(data, 'bath', ["baños", "baño"])

# Mostrar el DataFrame resultante
data

,Title,Price,Barrio,hab,m2,bath,charact1,charact2,charact3
0,"Piso de tres habitaciones Segura Lago, Sant Is...",€ 159.000,sant-isidre-patraix-favara-vara-de-quart,3 habitaciones\r,91 m²\r,1 baño\r,Planta 3\r,Ascensor\r,Balcón
1,"Piso de tres habitaciones Gremis Dels, Sant Is...",€ 245.000,sant-isidre-patraix-favara-vara-de-quart,3 habitaciones\r,132 m²\r,2 baños\r,Ascensor\r,Balcón\r,Terraza
2,"Ático Calle CERAMISTA ROS, Vara de Quart, Valè...",€ 210.000,sant-isidre-patraix-favara-vara-de-quart,2 habitaciones\r,55 m²\r,1 baño\r,Planta 8\r,Ascensor,NaN
3,"Piso tercera planta, Patraix, València",€ 400.000,sant-isidre-patraix-favara-vara-de-quart,5 habitaciones\r,151 m²\r,2 baños\r,Planta 3\r,Ascensor\r,Balcón
4,"Ático 115 m² con terraza, La Raiosa, València",€ 112.500,sant-isidre-patraix-favara-vara-de-quart,1 habitación\r,115 m²\r,1 baño\r,Planta 5\r,Balcón\r,Terraza
...,...,...,...,...,...,...,...,...,...
1179,"Piso de cuatro habitaciones buen estado, Penya...",€ 1.200.000,camins-al-grau,4 habitaciones\r,175 m²\r,3 baños\r,Lujo\r,Ascensor\r,Terraza
1180,"Piso Calle Padre Tomás Montañana 10, Camí Fond...","€ 215.000\r\n€ 230.000(-6,5%)",camins-al-grau,3 habitaciones\r,86 m²\r,2 baños\r,Planta 4\r,Ascensor\r,Balcón
1181,"Piso de tres habitaciones 86 m², Camí Fondo, V...",€ 230.000,camins-al-grau,3 habitaciones\r,86 m²\r,2 baños\r,Planta 4\r,Ascensor\r,Balcón
1182,"Piso de dos habitaciones séptima planta, Aiora...",€ 182.500,camins-al-grau,2 habitaciones\r,80 m²\r,2 baños\r,Planta 7\r,Ascensor,NaN


In [14]:
# Función para eliminar las palabras
def eliminar_palabras(texto, palabras_a_eliminar):
    for palabra in palabras_a_eliminar:
        texto = texto.replace(palabra, '')
    return texto



# Aplicar la función a la columna 'descripcion'
data['hab'] = data['hab'].apply(eliminar_palabras, args = (['habitación\r', 'habitaciones\r'],))
data['bath'] = data['bath'].apply(eliminar_palabras, args = (['baño\r', 'baños\r'],))
data['m2'] = data['m2'].apply(eliminar_palabras, args = (['m²\r'],))

,Title,Price,Barrio,hab,m2,bath,charact1,charact2,charact3
0,"Piso de tres habitaciones Segura Lago, Sant Is...",€ 159.000,sant-isidre-patraix-favara-vara-de-quart,3,91,1,Planta 3\r,Ascensor\r,Balcón
1,"Piso de tres habitaciones Gremis Dels, Sant Is...",€ 245.000,sant-isidre-patraix-favara-vara-de-quart,3,132,2,Ascensor\r,Balcón\r,Terraza
2,"Ático Calle CERAMISTA ROS, Vara de Quart, Valè...",€ 210.000,sant-isidre-patraix-favara-vara-de-quart,2,55,1,Planta 8\r,Ascensor,NaN
3,"Piso tercera planta, Patraix, València",€ 400.000,sant-isidre-patraix-favara-vara-de-quart,5,151,2,Planta 3\r,Ascensor\r,Balcón
4,"Ático 115 m² con terraza, La Raiosa, València",€ 112.500,sant-isidre-patraix-favara-vara-de-quart,1,115,1,Planta 5\r,Balcón\r,Terraza
...,...,...,...,...,...,...,...,...,...
1179,"Piso de cuatro habitaciones buen estado, Penya...",€ 1.200.000,camins-al-grau,4,175,3,Lujo\r,Ascensor\r,Terraza
1180,"Piso Calle Padre Tomás Montañana 10, Camí Fond...","€ 215.000\r\n€ 230.000(-6,5%)",camins-al-grau,3,86,2,Planta 4\r,Ascensor\r,Balcón
1181,"Piso de tres habitaciones 86 m², Camí Fondo, V...",€ 230.000,camins-al-grau,3,86,2,Planta 4\r,Ascensor\r,Balcón
1182,"Piso de dos habitaciones séptima planta, Aiora...",€ 182.500,camins-al-grau,2,80,2,Planta 7\r,Ascensor,NaN


In [16]:
data

,Title,Price,Barrio,hab,m2,bath,charact1,charact2,charact3
0,"Piso de tres habitaciones Segura Lago, Sant Is...",€ 159.000,sant-isidre-patraix-favara-vara-de-quart,3,91,1,Planta 3\r,Ascensor\r,Balcón
1,"Piso de tres habitaciones Gremis Dels, Sant Is...",€ 245.000,sant-isidre-patraix-favara-vara-de-quart,3,132,2,Ascensor\r,Balcón\r,Terraza
2,"Ático Calle CERAMISTA ROS, Vara de Quart, Valè...",€ 210.000,sant-isidre-patraix-favara-vara-de-quart,2,55,1,Planta 8\r,Ascensor,NaN
3,"Piso tercera planta, Patraix, València",€ 400.000,sant-isidre-patraix-favara-vara-de-quart,5,151,2,Planta 3\r,Ascensor\r,Balcón
4,"Ático 115 m² con terraza, La Raiosa, València",€ 112.500,sant-isidre-patraix-favara-vara-de-quart,1,115,1,Planta 5\r,Balcón\r,Terraza
...,...,...,...,...,...,...,...,...,...
1179,"Piso de cuatro habitaciones buen estado, Penya...",€ 1.200.000,camins-al-grau,4,175,3,Lujo\r,Ascensor\r,Terraza
1180,"Piso Calle Padre Tomás Montañana 10, Camí Fond...","€ 215.000\r\n€ 230.000(-6,5%)",camins-al-grau,3,86,2,Planta 4\r,Ascensor\r,Balcón
1181,"Piso de tres habitaciones 86 m², Camí Fondo, V...",€ 230.000,camins-al-grau,3,86,2,Planta 4\r,Ascensor\r,Balcón
1182,"Piso de dos habitaciones séptima planta, Aiora...",€ 182.500,camins-al-grau,2,80,2,Planta 7\r,Ascensor,NaN


In [17]:
pd.DataFrame.to_csv(data, "data/WS_Indomio.csv", index = False)